---

### <u>Initial Setup and Package Imports</u>

In [44]:
# Used to acquire and process HTML pages
import requests
from bs4 import BeautifulSoup
import re

# Used to store data and perform operations
import pandas as pd
import numpy as np

# Progress bar for current status
# from progressbar import ProgressBar

NUM_POKEMON = 1
START_URL = "http://bulbapedia.bulbagarden.net/wiki/Bulbasaur_(Pok%C3%A9mon)"

### <u>Define Functions</u>

In [45]:
#
def getName():
    pokemon_name = soup.find('big').find_next('b').text
    return pokemon_name

#
def getAbilities():
    abilities_table = soup.find(
        'a', {'title': 'Ability'}).parent.find_next('table')
    abilities = abilities_table.find_all(
        'td', {'style': lambda x: x != 'display: none'})
    return [ability.find('span').text for ability in abilities]

#
def get_next_pokemon_url():
    next_pokemon_url_table = soup.find_all('td', style='text-align: left')
    next_pokemon_url = "https://bulbapedia.bulbagarden.net" + \
        next_pokemon_url_table[0].find('a', href=True)['href']
    return next_pokemon_url

#
def get_evolves_into_or_from(tofrom):
    global current_url
    if 'Eevee' in current_url and tofrom == 'into':
        evolution_table = soup.find(
            'span', {'id': 'Evolution'}).parent.find_next('table')
 
        names = evolution_table.find_all(
            'a', {'title': re.compile('.*\(Pokémon\)')})
        evolves_data = []
        for name in names:
            evolves_data.append(name.text)
        return evolves_data
 
    evolution_p = soup.find('div', {'id': 'bodyContent'}).find_all('p')[1]
    for a in evolution_p('a'):
        a.replace_with_children()
    evolution_text = evolution_p.text
    evolves_data = []
 
    for evolves in evolution_text.split('evolves {} '.format(tofrom))[1:]:
        evolves_data.append(evolves.split(' ')[0].rstrip())
    if tofrom == 'from' and 'final form of' in evolution_text:
        evolves_data = [evolution_text.split(
            'final form of ')[-1][:-2]] + evolves_data
    return evolves_data

### <u>Generate Panda Series</u>

In [46]:
#
def generateSeries():
    series = pd.Series(
        data=[
            getName(),
            getAbilities()
        ],
        index=[
            'name',
            'abilities'
        ])
    return series

### <u>Create DataFrame</u>

In [47]:
#
def generateDF(pokemon_count):
    global soup, current_url
    df = pd.DataFrame()
    current_url = START_URL
    current_index = 1
    
    while current_index <= current_index:
        page = requests.get(current_url)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        series = generateSeries()
        df = df.append(series,ignore_index = True)
        
        current_index += 1
        current_url = get_next_pokemon_url()
        
    df.set_index('pokedex_number', inplace=True)
    df.index = df.index.astype('int64')
    df.index.name = 'pokedex_number'
    df = set_column_dtypes(df)
 
    # Pokemon's tiny image doesn't appear on its own page, but the next or
    # previous page, so roll this column back by 1 to set them correctly
    df.tiny_image_url = np.roll(df.tiny_image_url, -1)
 
    return df
 
df = generateDF(NUM_POKEMON)
df.to_pickle("./pokemon_dataframe.pkl")

KeyboardInterrupt: 